# Jet Energy Corrections

From exercise 2, recall the comparison of PFJets with GenJets:

In [6]:
import ROOT
f = ROOT.TFile("$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/ttjets.root")

h_ptAK4   = f.Get("h_ptAK4")
h_ptAK4Gen   = f.Get("h_ptAK4Gen")

h_ptAK4Gen.SetLineStyle(2) 
h_ptAK4Gen.SetLineColor(2) 
c_pt = ROOT.TCanvas('c_pt', 'c_pt', 600, 400)
ROOT.gPad.SetLogy()
h_ptAK4.Draw()
h_ptAK4Gen.Draw("same")
h_ptAK4.GetXaxis().SetRangeUser(0, 600)
leg = ROOT.TLegend(0.6, 0.6, 0.8, 0.8)
leg.AddEntry(h_ptAK4, "RECO", "l")
leg.AddEntry(h_ptAK4Gen, "GEN", "l")
leg.SetFillColor(0)
leg.SetLineColor(0)
leg.Draw("same")
ROOT.enableJSVis()
c_pt.Draw()


AttributeError: 'TObject' object has no attribute 'SetLineStyle'

Warning in <TFile::Init>: file /home/cms.dryu/CMSSW_10_6_6/src/Analysis/JMEDAS/notebooks/files/ttjets.root has no keys


The $p_{\mathrm{T}}$ distributions disagree quite a bit between the GenJets and PFJets. We need to apply the *jet energy corrections* (JECs), a sequence of corrections that address non-uniform responses in $p_{\mathrm{T}}$ and $\eta$, as well as an average correction for pileup. The JECs are often updated fairly late in the analysis cycle, simply due to the fact that the JEC experts start deriving the JECs at the same time the analyzers start developing their analyses. For this reason, it is imperative for analyzers to maintain flexibility in the JEC, and the software reflects this. It is possible to run the JEC software "on the fly" after you've done your heavy processing (PAT-tuple creation, skimming,etc). We will now show how this is done.

For more information and technical details on the jet energy scale calibration in CMS, look at the following twiki: https://twiki.cern.ch/twiki/bin/view/CMSPublic/WorkBookJetEnergyCorrections. 

Start by running the histogram-making code, this time asking it to apply the JECs. While it's running, take a look at the code and make sure you understand the parts relevant to JEC (try a text search for "args.correctJets"). 

In [ ]:
### RUN THIS CELL ONLY IF YOU ARE USING SWAN 
import os

##### REMEMBER TO MANUALLY COPY THE PROXY TO YOUR CERNBOX FOLDER AND TO MODIFY THE NEXT LINE
os.environ['X509_USER_PROXY'] = '/eos/home-X/Y/tmp/x509up_u0000'
if os.path.isfile(os.environ['X509_USER_PROXY']): pass
else: print("os.environ['X509_USER_PROXY'] ",os.environ['X509_USER_PROXY'])
os.environ['X509_CERT_DIR'] = '/cvmfs/cms.cern.ch/grid/etc/grid-security/certificates'
os.environ['X509_VOMS_DIR'] = '/cvmfs/cms.cern.ch/grid/etc/grid-security/vomsdir'

In [7]:
%%bash
python $CMSSW_BASE/src/Analysis/JMEDAS/scripts/jmedas_make_histograms.py --files=$CMSSW_BASE/src/Analysis/JMEDAS/data/MiniAODs/RunIIFall17MiniAODv2/ttjets.txt --outname=$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/ttjets_corr.root --maxevents=2000 --maxjets=6 --maxFiles 10 --correctJets Fall17_17Nov2017_V32_MC


Added root://cmsxrootd.fnal.gov//store/mc/RunIIFall17MiniAODv2/TTJets_TuneCP5_13TeV-madgraphMLM-pythia8/MINIAODSIM/PU2017_12Apr2018_94X_mc2017_realistic_v14-v1/20000/EA397089-9260-E811-8C95-4C79BA3201D5.root
Added root://cmsxrootd.fnal.gov//store/mc/RunIIFall17MiniAODv2/TTJets_TuneCP5_13TeV-madgraphMLM-pythia8/MINIAODSIM/PU2017_12Apr2018_94X_mc2017_realistic_v14-v1/70000/BC2CF28A-E286-E811-A7C3-0CC47A7FC6F8.root
Added root://cmsxrootd.fnal.gov//store/mc/RunIIFall17MiniAODv2/TTJets_TuneCP5_13TeV-madgraphMLM-pythia8/MINIAODSIM/PU2017_12Apr2018_94X_mc2017_realistic_v14-v1/70000/6ED8EEC0-EB88-E811-97DD-FA163E933851.root
Added root://cmsxrootd.fnal.gov//store/mc/RunIIFall17MiniAODv2/TTJets_TuneCP5_13TeV-madgraphMLM-pythia8/MINIAODSIM/PU2017_12Apr2018_94X_mc2017_realistic_v14-v1/20000/DCFEEE83-AD65-E811-B55A-24BE05CEEB81.root
Added root://cmsxrootd.fnal.gov//store/mc/RunIIFall17MiniAODv2/TTJets_TuneCP5_13TeV-madgraphMLM-pythia8/MINIAODSIM/PU2017_12Apr2018_94X_mc2017_realistic_v14-v1/70000/26

The string "Fall17_17Nov2017_V32" points to the JEC text files in the directory /data/JECs, which were downloaded from [this twiki](https://twiki.cern.ch/twiki/bin/view/CMS/JECDataMC). You can also create the text files using the JetCorrectorDBReader module in CMSSW (see the cmsRun cfg at scripts/JetCorrectionDBReader_cfg.py), but for technical reasons, this method can be error-prone (see the JEC twiki for more details). 

## Exercise: Before and after JECs

Let's check the GenJets-PFJets agreement after applying the JECs:

In [3]:
f_corr = ROOT.TFile("$CMSSW_BASE/src/Analysis/JMEDAS/notebooks/files/ttjets_corr.root")

h_ptAK4_corr = f_corr.Get("h_ptAK4")
h_ptAK4Gen_corr = f_corr.Get("h_ptAK4Gen")

h_ptAK4Gen_corr.SetLineStyle(2) 
h_ptAK4Gen_corr.SetLineColor(2) 
c_corr = ROOT.TCanvas('c_corr', 'c', 800, 400)
c_corr.Divide(2,1)
c_corr.cd(1)
ROOT.gPad.SetLogy()
h_ptAK4_corr.Draw()
h_ptAK4_corr.SetTitle("AK4 Jet p_{T} (corrected)")
h_ptAK4Gen_corr.Draw("same")
h_ptAK4_corr.GetXaxis().SetRangeUser(0, 1000)
leg_corr = ROOT.TLegend(0.6, 0.6, 0.8, 0.8)
leg_corr.AddEntry(h_ptAK4, "RECO", "l")
leg_corr.AddEntry(h_ptAK4Gen, "GEN", "l")
leg_corr.SetFillColor(0)
leg_corr.SetLineColor(0)
leg_corr.Draw("same")

c_corr.cd(2)
ROOT.gPad.SetLogy()
h_ptAK4.Draw()
h_ptAK4.SetTitle("AK4 Jet p_{T} (uncorrected)")
h_ptAK4Gen.Draw("same")
h_ptAK4.GetXaxis().SetRangeUser(0, 1000)
leg = ROOT.TLegend(0.6, 0.6, 0.8, 0.8)
leg.AddEntry(h_ptAK4, "RECO", "l")
leg.AddEntry(h_ptAK4Gen, "GEN", "l")
leg.SetFillColor(0)
leg.SetLineColor(0)
leg.Draw("same")

c_corr.Draw()

c_compare = ROOT.TCanvas("c_compare", "c_compare", 400, 400)
c_compare.SetLogy()
h_ptAK4_norm = h_ptAK4.Clone()
h_ptAK4_norm.Scale(1. / h_ptAK4_norm.Integral())
h_ptAK4_norm.SetLineStyle(2)
h_ptAK4_norm.SetLineColor(ROOT.kRed)
h_ptAK4_norm.SetTitle("Corrected vs. Uncorrected")
h_ptAK4_norm.Draw("hist")

h_ptAK4_corr_norm = h_ptAK4_corr.Clone()
h_ptAK4_corr_norm.Scale(1. / h_ptAK4_corr_norm.Integral())
h_ptAK4_corr_norm.SetLineStyle(1)
h_ptAK4_corr_norm.SetLineColor(ROOT.kBlack)
h_ptAK4_corr_norm.Draw("hist same")
l_compare = ROOT.TLegend(0.55, 0.5, 0.88, 0.8)
l_compare.SetFillColor(0)
l_compare.SetBorderSize(0)
l_compare.AddEntry(h_ptAK4_norm, "Uncorrected AK4PFJets", "l")
l_compare.AddEntry(h_ptAK4_corr_norm, "Corrected AK4PFJets", "l")
l_compare.Draw()
c_compare.Draw()



## JEC Uncertainties

Since we've applied the JEC corrections to the distributions, we should also assign a systematic uncertainty to the procedure. The procedure is explained at [this twiki](https://twiki.cern.ch/twiki/bin/view/CMSPublic/WorkBookJetEnergyCorrections#JetCorUncertainties). 

The uncertainties are implemented in `jmedas_make_histograms.py`. The histogram files you've made already have the up- and down-variation histograms. The relevant piece of code is:

```
corr = 1.0
corrUp = 1.0
corrDn = 1.0
# Get the latest, greatest jet corrections
if args.correctJets : 
    jec.setJetEta( uncorrJet.eta() )
    jec.setJetPt ( uncorrJet.pt() )
    jec.setJetE  ( uncorrJet.energy() )
    jec.setJetA  ( jet.jetArea() )
    jec.setRho   ( rhoValue[0] )
    jec.setNPV   ( len(pvs) )
    icorr = jec.getCorrection()
    corr *= icorr
    corrUp *= icorr
    corrDn *= icorr


    #JEC Uncertainty
    jecUnc.setJetEta( uncorrJet.eta() )
    jecUnc.setJetPhi( uncorrJet.phi() )
    jecUnc.setJetPt( corr * uncorrJet.pt() )
    corrUp += jecUnc.getUncertainty(1)
    jecUnc.setJetEta( uncorrJet.eta() )
    jecUnc.setJetPhi( uncorrJet.phi() )
    jecUnc.setJetPt( corr * uncorrJet.pt() )
    corrDn -= jecUnc.getUncertainty(0)


h_ptAK4.Fill( corr * uncorrJet.pt() )
h_JECValueAK4.Fill( corr )
h_ptUncorrAK4.Fill( uncorrJet.pt() )
h_ptDownAK4.Fill( corrDn * uncorrJet.pt() )
h_ptUpAK4.Fill( corrUp * uncorrJet.pt() )
```

Run the next cell to plot a comparison of the nominal and varied histograms.

In [4]:
h_ptAK4_corr   = f_corr.Get("h_ptAK4")
h_ptAK4_corrUp  = f_corr.Get("h_ptUpAK4")
h_ptAK4_corrDown  = f_corr.Get("h_ptDownAK4")

h_ptAK4_corr.SetLineWidth(2)
h_ptAK4_corrUp.SetLineStyle(2)
h_ptAK4_corrUp.SetLineColor(ROOT.kGreen+1) 
h_ptAK4_corrDown.SetLineStyle(2)
h_ptAK4_corrDown.SetLineColor(ROOT.kRed) 
h_ptAK4_corrUp.SetLineWidth(2)
h_ptAK4_corrDown.SetLineWidth(2)

c_JECunc = ROOT.TCanvas('c', 'c')

h_ptAK4_corr.Draw()
h_ptAK4_corrUp.Draw("same")
h_ptAK4_corrDown.Draw("same")
h_ptAK4_corr.GetXaxis().SetRangeUser(0, 400)

leg_corr = ROOT.TLegend(0.45, 0.55, 0.75, 0.8)
leg_corr.AddEntry(h_ptAK4_corr, "Nominal JEC", "l")
leg_corr.AddEntry(h_ptAK4_corrUp, "JEC +1 #sigma", "l")
leg_corr.AddEntry(h_ptAK4_corrDown, "JEC -1 #sigma", "l")
leg_corr.SetLineWidth(0)
leg_corr.SetFillColor(0)
leg_corr.SetShadowColor(0)
leg_corr.Draw()

c_JECunc.Draw()


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c


## Discussion
Why do we need to calibrate jet energy? Why is "jet response" not equal to 1? Can you think of a physics process in nature that can help us calibrate the jet response to 1?

The amount of material in front of the CMS calorimeter varies by $\eta$. Therefore, the calorimeter response to jet is also a function of jet $\eta$. Can you think of a physics process in nature that can help us calibrate the jet response in $\eta$ to be uniform ?
